# Amazon FBA Taxes 2021

In [1]:
# Set working directory
%cd "D:\Amazon FBA\businesstracker"

from amz_transactions import Transactions, Cashflows
from amz_charting import generate_cashflow_graph
import pandas as pd
import numpy as np


D:\Amazon FBA\businesstracker


In [2]:
# Connect to PSQL and create Transactions object
db = Transactions(configfile='psql_businessfinances.ini')

## Update Recieved Inventory in SQL from Inventory Reports

In [3]:
# df_inv = pd.read_csv("inventory_recieved.csv")
# df_inv["date"] = pd.to_datetime(df_inv["date"])
# for row in df_inv.itertuples():
#     query = f"INSERT INTO inventory_recieved VALUES ({row[0]},'{row[1]}','{row[2]}','{row[3]}','{row[4]}',{row[5]},'{row[6]}','{row[7]}',{row[8]}) RETURNING *"
#     db.execute_command(query)

## Retrieve datasets

In [4]:
# Retrieve all transactions from PSQL
query = f'''
        SELECT *
        FROM amz_transactions
        ORDER BY posted_date_time
        '''
df_tr = pd.read_sql(query,db.conn)
df_inv = pd.read_sql(f"SELECT * FROM inventory_recieved ORDER BY date",db.conn)
df_orders = pd.read_sql(f"SELECT DISTINCT order_id,posted_date_time FROM amz_transactions ORDER BY posted_date_time",db.conn)

# Load FXUSDCAD from BoC CSV
df_fx = pd.read_csv("FXUSDCAD-2021.csv")


In [5]:
## Helper function
def window_helper(group,op,col):
    group = group.sort_values('date')
    nam_col = op+col
    if op == "Sum":
        group[nam_col] = group[col].expanding(1).sum()
    elif op == "Avg":
        group[nam_col] = group[col].expanding(1).mean()
    return group

In [6]:
# Cleanup datasets

df_fx["date"] = pd.to_datetime(df_fx["date"])
df_fx["FXUSDCAD"] = df_fx["FXUSDCAD"].astype(float)
df_fx = df_fx.set_index("date").asfreq(freq='D',method="ffill").reset_index()

df_tr['date'] = df_tr["posted_date_time"].dt.date
df_tr['date'] = pd.to_datetime(df_tr['date'])

df_inv.rename(columns={"quantity":"inv_rec"}, inplace=True)
df_inv['date'] = pd.to_datetime(df_inv['date'])
# Convert random +1 and -1 inventory to 0 as they are not real entries
df_inv.loc[(df_inv["inv_rec"] >= -1) & (df_inv["inv_rec"] <= 1),"inv_rec"] = 0
df_inv = df_inv.merge(df_fx, how="left", on="date")
df_inv["CPU_USD"] = df_inv["cost"]/df_inv["inv_rec"]
df_inv["CPU"] = df_inv["CPU_USD"] * df_inv["FXUSDCAD"]

In [7]:
### Merge in inventory orders
df_merged = df_tr.merge(df_inv[["date","inv_rec","CPU"]], how="outer", on="date")
df_merged["inv_rec"].fillna(0, inplace = True)
df_merged["CPU"].fillna(0, inplace = True)
# Cleanup CPU, first forward fill then set the initial NaNs to 0
df_merged["CPU"] = df_merged["CPU"].fillna(method="ffill").fillna(0)
# Remove duplicated inv_rec and CPU due to merge. Ensures inventory only added once
df_merged.loc[df_merged.duplicated(["date","inv_rec"]),"inv_rec"] = 0
df_merged.loc[df_merged.duplicated(["date","CPU"]),"CPU"] = 0

### Resort by dates to align everything
df_merged = df_merged.sort_values('date')
df_merged["quantity_purchased"] = df_merged["quantity_purchased"].fillna(0)

### Find the change in inventory by sales or refunds
df_merged["delta_inv"] = 0
df_merged.loc[df_merged["transaction_type"]=="Order","delta_inv"] = -1*df_merged["quantity_purchased"]
df_merged.loc[df_merged["transaction_type"]=="Refund","delta_inv"] = 1
# Remove duplicated due to merge. Ensures salers/refunds only added once per order
df_merged.loc[df_merged.duplicated(["order_id","delta_inv"]),"delta_inv"] = 0

### Clenaup index
df_merged = df_merged.reset_index(drop=True)

### Calculate current inventory based on inventory recieving and sales/refunds
df_merged["curr_inv"] = df_merged["inv_rec"] + df_merged["delta_inv"]
df_merged["curr_inv"] = df_merged["curr_inv"].expanding(1).sum()


In [8]:
## Calculate ACB 
# Note that CPU is in CAD
acb = np.zeros(len(df_merged))
for i in range(1,len(acb)):
    if max(0,df_merged.iloc[i]["inv_rec"]) > 0: # Increase ACB only if inventory increased
        curr_inv = df_merged.iloc[i]["curr_inv"]-df_merged.iloc[i]["inv_rec"]
        new_inv = df_merged.iloc[i]["inv_rec"]
        new_cost = new_inv*df_merged.iloc[i]["CPU"]
        curr_cost = curr_inv*acb[i-1]
        acb[i] = (new_cost+curr_cost)/(new_inv+curr_inv)
    else: # Else keep previous ACB
        acb[i] = acb[i-1]

df_merged["acb"] = acb

In [9]:
# Convert USD to CAD
# Applies to amount only; ACB already in CAD as CPU used is in CAD abouve
df_merged = df_merged.merge(df_fx, how="left", on="date")
df_merged = df_merged.rename(columns={"amount":"amount_USD"})
df_merged["amount"] = df_merged["amount_USD"] * df_merged["FXUSDCAD"]

In [10]:
# Export CSV
df_merged.sort_values(["date","posted_date_time"]).to_csv("merged.csv")

All data is prepared for analysis. Completed:

    1. Inventory recieving
    2. ACB calculation
    3. Inventory deltas for sales/refunds
    4. Determined daily FX-USDCAD rates

## Analysis

In [11]:
# Setup
start = pd.to_datetime('2021-01-01')
end = pd.to_datetime('2022-01-01')

mask = (df_merged["date"] >= start) & (df_merged["date"] < end)
df_merged = df_merged[mask]

In [19]:
# Sales Revenue
# For every order_id, get amount_type == "ItemPrice" and amount_description == "Principal"
# Includes the revenue subtraction due to refunds
mask = (df_merged["amount_type"] == "ItemPrice") & (df_merged["amount_description"] == "Principal")
revenue = df_merged[mask]["amount"].sum()
print("Sales Revenue = $", round(abs(revenue),2))

Sales Revenue = $ 10848.42


In [18]:
# Advertising Expense
# PPC Campaign: Sum where amount_type == "Cost of Advertising"
# Coupons: Sum where transaction_type == "CouponRedemptionFee" + 
#          Sum where amount_type == "Promotion" and amount_description == "Principal"
ppc_mask = df_merged["amount_type"] == "Cost of Advertising"
coupred_mask = df_merged["transaction_type"] == "CouponRedemptionFee of Advertising"
coupro_mask = (df_merged["amount_type"] == "Promotion") & (df_merged["amount_description"] == "Principal")

advert = df_merged[ppc_mask]["amount"].sum()
advert = advert + df_merged[coupred_mask]["amount"].sum() 
advert = advert + df_merged[coupro_mask]["amount"].sum()
print("Cost of Advertising = $", round(abs(advert),2))

Cost of Advertising = $ 4870.66


In [20]:
# FBA Fees
# Commissions: Sum where amount_type == "ItemFees" and amount_description == "Commission" + 
#              Sum where amount_type == "ItemFees" and amount_description == "FBAPerUnitFulfillmentFee" + 
#              Sum where amount_type == "ItemFees" and amount_description == "RefundCommission" + 
# Includes commissions refunded in Refunds from original order

comm_mask = (df_merged["amount_type"] == "ItemFees") & (df_merged["amount_description"] == "Commission")
fullf_mask = (df_merged["amount_type"] == "ItemFees") & (df_merged["amount_description"] == "FBAPerUnitFulfillmentFee")
refcom_mask = (df_merged["amount_type"] == "ItemFees") & (df_merged["amount_description"] == "RefundCommission")

fees = df_merged[comm_mask]["amount"].sum()
fees = fees + df_merged[fullf_mask]["amount"].sum()
fees = fees + df_merged[refcom_mask]["amount"].sum() 
print("FBA Fees = $", round(abs(fees),2))

FBA Fees = $ 2568.0


In [21]:
# Storage Fees
# Sum where amount_description == "Storage Fee"]
store_mask = df_merged["amount_description"] == "Storage Fee"
store = df_merged[comm_mask]["amount"].sum()
print("Storage Fees = $", round(abs(store),2))

Storage Fees = $ 1564.78


In [22]:
# Cost of Goods Sold
# Sum of quantity sold * ACB 
# Refunds are taken care when groupedby order_id and the delta_inv = 0 since refunds are under same order_id as order
df_temp = df_merged.groupby("order_id",as_index=False).sum()
df_temp["acb"] = df_temp["acb"]/df_temp["quantity_purchased"] # Re-normalize ACB after grouping
cogs = (df_temp["acb"] * df_temp["delta_inv"]).sum()
print("COGS = $", round(abs(cogs),2))

COGS = $ 2472.77


In [23]:
# Subscription costs
# Sum where amount_description == "Subscription Fee"]
sub_mask = df_merged["amount_description"] == "Subscription Fee"
sub_cost = df_merged[sub_mask]["amount"].sum()
print("Subscription Costs = $", round(abs(sub_cost),2))

Subscription Costs = $ 552.6
